In [18]:
#this code works only if there is only one "white spot" on the mask 

import cv2
import numpy as np

def find_longest_diameter(mask_image):
    # Find contours of the lesion in the mask
    contours, _ = cv2.findContours(mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        print("Error: No contour found in the mask image.")
        return None
    
    # Find the convex hull of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    
    # Find the longest diameter within the convex hull
    longest_diameter = find_longest_line(hull)
    
    return longest_diameter

def find_longest_line(contour):
    # Initialize variables to store the longest line and its length
    max_length = 0
    longest_line = None
    
    # Iterate through all combinations of points to find the longest line segment
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            # Calculate the length of the line segment
            length = np.linalg.norm(contour[i] - contour[j])
            
            # Update the longest line if the current line segment is longer
            if length > max_length:
                max_length = length
                longest_line = (contour[i][0], contour[j][0])
    
    return longest_line

def rotate_image(image, angle):
    # Get the dimensions of the image
    h, w = image.shape[:2]
    
    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    
    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    
    return rotated_image

def crop_to_sides(mask_image, longest_diameter):
    # Determine the angle of rotation to make the longest diameter vertical
    angle = np.degrees(np.arctan2(longest_diameter[1][1] - longest_diameter[0][1], longest_diameter[1][0] - longest_diameter[0][0]))
    
    # If the angle is more than 90 degrees, subtract 180 degrees to make it negative
    if angle > 90:
        angle -= 180
    
    # Rotate the mask image
    rotated_mask = rotate_image(mask_image, angle)
    
    # Find contours of the rotated lesion
    contours, _ = cv2.findContours(rotated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the bounding rectangle of the rotated lesion
    x, y, w, h = cv2.boundingRect(contours[0])
    
    # Crop the rotated mask image to include the left and right sides of the lesion
    cropped_mask = rotated_mask[y:y+h, x:x+w]
    
    return cropped_mask

# Example usage
if __name__ == "__main__":
    # Load the mask image
    mask_image = cv2.imread('PAT_492_1784_50_mask.png', cv2.IMREAD_GRAYSCALE)
    
    # Ensure mask image is loaded correctly
    if mask_image is None:
        print("Error: Failed to load the mask image.")
    else:
        # Find the longest diameter of the lesion
        longest_diameter = find_longest_diameter(mask_image)
        
        if longest_diameter is not None:
            # Crop the lesion image to the left and right sides divided by the longest diameter
            cropped_mask = crop_to_sides(mask_image, longest_diameter)
            
            # Display the cropped mask image
            cv2.imshow("Cropped Mask", cropped_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [17]:
import cv2
import numpy as np

def find_farthest_vertical_points(mask):
    # Znajdź współrzędne punktów białych na masce
    white_points = np.argwhere(mask == 255)

    max_y_diff = 0
    farthest_points = None

    # Przechowuj najdalsze dwa punkty
    farthest_two_points = [(0, 0), (0, 0)]

    # Przejdź przez każdą parę punktów białych i znajdź dwie najdalsze zmiany
    for point1 in white_points:
        for point2 in white_points:
            y_diff = abs(point2[0] - point1[0])
            if y_diff > max_y_diff:
                max_y_diff = y_diff
                farthest_two_points = [(point1[1], point1[0]), (point2[1], point2[0])]

    return farthest_two_points

def draw_vertical_line(image, points):
    # Narysuj linię pomiędzy dwoma punktami
    cv2.line(image, (points[0][0], points[0][1]), (points[1][0], points[1][1]), (255, 0, 0), 2)

# Wczytaj obraz z maską zmiany skórnej
mask_image = cv2.imread('PAT_715_4073_115_mask.png', cv2.IMREAD_GRAYSCALE)

# Znajdź dwie najdalej oddalone punkty w pionie
vertical_points = find_farthest_vertical_points(mask_image)

if vertical_points is not None:
    # Narysuj pionowe linie na oryginalnym obrazie
    original_image = cv2.imread('PAT_715_4073_115_mask.png')
    draw_vertical_line(original_image, vertical_points[0])
    draw_vertical_line(original_image, vertical_points[1])

    # Sprawdź, czy obraz został poprawnie wczytany przed wyświetleniem
    if original_image is not None and original_image.any():
        # Wyświetl wyniki
        cv2.imshow('Original Image with Vertical Lines', original_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Błąd: Obraz nie został poprawnie wczytany.")
else:
    print("Nie znaleziono wystarczającej liczby punktów białych na masce.")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\cieci\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\cieci\AppData\Local\Temp\ipykernel_10484\443469619.py", line 32, in <module>
    vertical_points = find_farthest_vertical_points(mask_image)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cieci\AppData\Local\Temp\ipykernel_10484\443469619.py", line -1, in find_farthest_vertical_points
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\cieci\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cieci\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return For